In [ ]:
using InspectDR
using Colors

#Define colour constants:
black = RGB24(0, 0, 0); white = RGB24(1, 1, 1)
red = RGB24(1, 0, 0); green = RGB24(0, 1, 0); blue = RGB24(0, 0, 1)

#When MIME"image/svg+xml" is disabled, Jupyter eventually requests PNG inline graphics:
#(SVG outputs do not render well in notebooks for some reason...)
InspectDR.defaults.rendersvg = false
nothing

# Define "PlotBode" function

In [ ]:
#Modify previously-generated Bode plot:
#fmin, fmax, p0: in Hz
function PlotBode(plot, fmin, fmax, p0)
    const style = :solid
    const npts = 100
    f = logspace(log10(fmin), log10(fmax), npts)
    y = 1./(1-(im*f/p0))
    plot.data = [] #Clear old data
    wfrm = add(plot, f, y)
    wfrm.line = line(color=blue, width=1, style=style)
    plot.ext = InspectDR.PExtents2D(NaN, NaN, -40, 0)
    plot.ext_full = deepcopy(plot.ext)
    return plot
end

#Create new Bode plot:
function PlotBode(fmin, fmax, p0)
    const w = 500; const h = w/1.6 #Select plot width/height
    plot = InspectDR.Plot2D()
    plot.axes = InspectDR.axes(:log10, :dB20)
    plot.layout.grid = grid(vmajor=true, vminor=true, hmajor=false)
    plot.layout.wdata=w; plot.layout.hdata=h
    a = plot.annotation
    a.title = "Bode Plot"
    a.xlabel = "Frequency (Hz)"
    a.ylabel = "Amplitude (dB)"
    return PlotBode(plot, fmin, fmax, p0)
end


# Setup: Interact, Reactive, and Signals
(Also create initial plot object)

In [ ]:
using Interact, Reactive

fmult = Signal(Float64, 1e9) #GHz
fmax = Signal(Float64, 20)
p0 = Signal(Float64, 5)
s = value(fmult) #scale factor
pobj = PlotBode(s*1e-3, s*value(fmax), s*value(p0)) #Create initial plot object
nothing

# Interact/Reactive Control: Inline Plots
Use sliders/Text boxes to change pole frequency

In [ ]:
#TODO: Cannot initialize togglebuttons.
w_fmult = togglebuttons([("kHz", 1e3), ("MHz", 1e6), ("GHz", 1e9)], value=value(fmult), signal=fmult)
display(w_fmult)

#TODO: Display fmax textbox.
txt_fmax = textbox(value(fmax), label="max freq", signal=fmax) #Will not display??
#txt_fmax = textbox(value(fmax), label="max freq") #This one does not cause hangups
#display(txt_fmax) #Will hang session, for some reason

sld_p0 = map((m)->(slider(1e-3:1e-3:m, value=value(p0), label="pole freq", signal=p0)), fmax)
display(sld_p0)

sig_plot = map((s, _fmax, _p0)->PlotBode(pobj, s*1e-3, _fmax*s, _p0*s), fmult, fmax, p0)
display(sig_plot)

nothing

# Interact/Reactive Control: Inspect/Gtk GUI
(Re-display GUI controls for convenience)

In [ ]:
gtkgui = display(InspectDR.GtkDisplay(), pobj) #Display plot in Gtk GUI.

#Refresh GUI when plot changes:
sig_gtkgui = map((p)->InspectDR.refresh(gtkgui), sig_plot)

#Re-display GUI controls, for convenience:
map(display, [w_fmult, sld_p0])
nothing

# Demo complete!